## 嵌入

在我們之前的例子中，我們使用了高維度的詞袋向量，其長度為 `vocab_size`，並且我們明確地將低維度的位置信息向量轉換為稀疏的獨熱表示。這種獨熱表示並不具備記憶效率，此外，每個詞都被獨立地處理，也就是說，獨熱編碼的向量無法表達詞與詞之間的語義相似性。

在本單元中，我們將繼續探索 **News AG** 數據集。首先，讓我們載入數據並從之前的筆記本中獲取一些定義。


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## 什麼是嵌入？

**嵌入**的概念是用低維度的密集向量來表示單詞，這些向量能夠在某種程度上反映單詞的語義。我們稍後會討論如何構建有意義的單詞嵌入，但現在我們可以將嵌入簡單地理解為一種降低單詞向量維度的方法。

因此，嵌入層會將一個單詞作為輸入，並生成指定 `embedding_size` 的輸出向量。從某種意義上說，它與 `Linear` 層非常相似，但不同的是，它不需要接受 one-hot 編碼的向量，而是可以直接接受單詞的編號作為輸入。

通過將嵌入層作為我們網絡的第一層，我們可以從詞袋模型（bag-of-words）切換到 **嵌入袋模型**（embedding bag model）。在這種模型中，我們首先將文本中的每個單詞轉換為對應的嵌入向量，然後對所有這些嵌入向量執行某種聚合函數，例如 `sum`、`average` 或 `max`。

![展示一個針對五個序列單詞的嵌入分類器的圖片。](../../../../../translated_images/zh-MO/embedding-classifier-example.b77f021a7ee67eee.webp)

我們的分類器神經網絡將以嵌入層開始，接著是聚合層，最後在其上添加一個線性分類器：


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### 處理變動的序列大小

由於這種架構，我們的網路需要以特定方式建立小批量資料。在前一單元中，使用詞袋模型（bag-of-words）時，小批量中的所有 BoW 張量都具有相同的大小 `vocab_size`，無論文本序列的實際長度如何。一旦我們改用詞嵌入（word embeddings），每個文本樣本中的詞數就會變得不固定，而在將這些樣本合併成小批量時，我們需要進行一些填充。

這可以透過向資料來源提供 `collate_fn` 函數的方式來完成：


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### 訓練嵌入分類器

現在我們已經定義了合適的資料加載器，我們可以使用上一單元中定義的訓練函數來訓練模型：


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

**注意**：我們在這裡僅訓練 25k 筆記錄（少於一個完整的 epoch）以節省時間，但您可以繼續訓練，編寫一個函數來進行多個 epoch 的訓練，並嘗試調整學習率參數以獲得更高的準確率。您應該能夠達到約 90% 的準確率。


### EmbeddingBag 層與變長序列表示法

在之前的架構中，我們需要將所有序列填充（pad）到相同的長度，才能將它們放入一個小批次中。這並不是表示變長序列最有效率的方法——另一種方法是使用 **offset** 向量，該向量會保存所有序列在一個大型向量中的偏移量。

![顯示偏移序列表示法的圖片](../../../../../translated_images/zh-MO/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **注意**：在上圖中，我們展示的是一個字符序列，但在我們的例子中，我們處理的是單詞序列。然而，使用偏移向量表示序列的基本原則是相同的。

為了使用偏移表示法，我們使用 [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) 層。它與 `Embedding` 類似，但它接受內容向量和偏移向量作為輸入，並且還包含一個平均層，該層可以是 `mean`、`sum` 或 `max`。

以下是使用 `EmbeddingBag` 的修改後的網路：


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

要準備用於訓練的數據集，我們需要提供一個轉換函數來準備偏移向量：


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

注意，與之前的所有例子不同，我們的網絡現在接受兩個參數：數據向量和偏移向量，它們的大小不同。同樣，我們的數據加載器也提供了3個值而不是2個：文本和偏移向量都作為特徵提供。因此，我們需要稍微調整我們的訓練函數來處理這一點：


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## 語意嵌入：Word2Vec

在我們之前的例子中，模型的嵌入層學會了將單詞映射為向量表示，但這種表示並沒有太多語意上的意義。如果能學習到這樣的向量表示，讓相似的單詞或同義詞在某種向量距離（例如歐幾里得距離）上彼此接近，那就更好了。

為了實現這一點，我們需要以特定的方式在大量文本上預訓練嵌入模型。最早的語意嵌入訓練方法之一被稱為 [Word2Vec](https://en.wikipedia.org/wiki/Word2vec)。它基於兩種主要架構，用於生成單詞的分佈式表示：

- **連續詞袋模型** (CBoW) —— 在這種架構中，我們訓練模型根據周圍的上下文來預測一個單詞。給定 ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$，模型的目標是從 $(W_{-2},W_{-1},W_1,W_2)$ 預測 $W_0$。
- **連續跳字模型** (Skip-Gram) 則與 CBoW 相反。模型使用周圍的上下文窗口單詞來預測當前單詞。

CBoW 的訓練速度較快，而 Skip-Gram 雖然較慢，但在表示不常見單詞方面表現更好。

![顯示 CBoW 和 Skip-Gram 算法如何將單詞轉換為向量的圖片。](../../../../../translated_images/zh-MO/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

為了試驗在 Google News 數據集上預訓練的 Word2Vec 嵌入，我們可以使用 **gensim** 庫。以下是找到與 'neural' 最相似的單詞的示例：

> **注意：** 當你第一次創建單詞向量時，下載它們可能需要一些時間！


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


我們也可以從詞語計算向量嵌入，用於訓練分類模型（為了清楚起見，我們僅顯示向量的前20個組件）：


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

語義嵌入的優點在於可以操控向量編碼來改變語義。例如，我們可以要求找到一個詞，其向量表示盡可能接近詞 *國王* 和 *女人*，並且盡可能遠離詞 *男人*：


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBoW 和 Skip-Grams 都是「預測型」嵌入，因為它們只考慮局部上下文。Word2Vec 並未利用全局上下文。

**FastText** 在 Word2Vec 的基礎上進一步改進，通過學習每個詞以及詞內字符 n-gram 的向量表示。在每次訓練步驟中，這些表示的值會被平均成一個向量。雖然這增加了預訓練的計算量，但它使得詞嵌入能夠編碼子詞信息。

另一種方法，**GloVe**，利用共現矩陣的概念，使用神經方法將共現矩陣分解為更具表達力且非線性的詞向量。

你可以通過更改嵌入模型為 FastText 和 GloVe 來試驗這些例子，因為 gensim 支援多種不同的詞嵌入模型。


## 在 PyTorch 中使用預訓練的嵌入

我們可以修改上述範例，將嵌入層中的矩陣預先填入語義嵌入，例如 Word2Vec。我們需要考慮到，預訓練嵌入的詞彙表和我們文本語料庫的詞彙表可能不完全匹配，因此我們會用隨機值初始化缺失詞彙的權重：


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


現在讓我們訓練模型。請注意，由於嵌入層的大小更大，因此參數的數量大幅增加，訓練模型所需的時間比前一個例子顯著增加。此外，正因如此，如果我們想避免過擬合，可能需要在更多的例子上訓練模型。


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

在我們的情況中，並未看到準確率有顯著提升，這可能是由於詞彙差異較大所致。  
為了解決詞彙差異的問題，我們可以採用以下解決方案之一：  
* 重新訓練 word2vec 模型以適應我們的詞彙  
* 使用預訓練 word2vec 模型的詞彙來載入我們的數據集。在載入數據集時，可以指定使用的詞彙。  

後者的方法似乎更簡單，尤其是因為 PyTorch 的 `torchtext` 框架內建了對嵌入的支持。我們可以，例如，以以下方式實例化基於 GloVe 的詞彙：  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


已載入的詞彙具有以下基本操作：
* `vocab.stoi` 字典允許我們將單詞轉換為其字典索引
* `vocab.itos` 則執行相反操作——將數字轉換為單詞
* `vocab.vectors` 是嵌入向量的陣列，因此要獲取單詞 `s` 的嵌入，我們需要使用 `vocab.vectors[vocab.stoi[s]]`

以下是一個操作嵌入的範例，用來展示方程式 **kind-man+woman = queen**（我稍微調整了係數以使其生效）：


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

要使用這些嵌入來訓練分類器，我們首先需要使用GloVe詞彙表對我們的數據集進行編碼：


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

正如我們上面所見，所有向量嵌入都存儲在 `vocab.vectors` 矩陣中。這使得通過簡單的複製將這些權重加載到嵌入層的權重中變得非常容易：


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

我們未能看到準確性顯著提高的原因之一是因為我們的數據集中的某些詞語在預訓練的GloVe詞彙表中缺失，因此它們基本上被忽略了。為了克服這一問題，我們可以在我們的數據集上訓練自己的嵌入。


## 語境嵌入

傳統預訓練嵌入表示（例如 Word2Vec）的一個主要限制是詞義消歧的問題。雖然預訓練嵌入能夠捕捉到一些詞語在語境中的含義，但每個詞的所有可能含義都被編碼到同一個嵌入中。這可能會在下游模型中引發問題，因為許多詞語（例如 "play"）的含義會根據使用的語境而有所不同。

例如，"play" 在以下兩個句子中的含義就完全不同：
- 我去看了一場**戲劇**。
- 約翰想和他的朋友一起**玩**。

上述的預訓練嵌入將 "play" 的這兩種含義表示為同一嵌入。為了克服這一限制，我們需要基於**語言模型**來構建嵌入。語言模型是在大量文本語料庫上訓練的，並且*了解*詞語如何在不同語境中組合使用。討論語境嵌入超出了本教程的範圍，但我們會在下一單元討論語言模型時回到這個主題。



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵信息，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
